In [160]:
import numpy as np
import pandas as pd

In [161]:
def str2array(x, delimter=' ', dtype=float):
    '''
    converts string of [23 21 0 12] to numpy array
    '''
    xs = x.strip()[1:-1].split(delimter)
    new_x = []
    for _x in xs:
        new_x.append(dtype(_x))
    return np.array(new_x)

def load_isaac_results(experiment, prefix, full_exeriment=True, experiments_dir = '../experiments'):
    '''
    Loads isaac data regardless of what experiment (full or gripper only).
    '''
    
    cats = []
    indcs = []
    env_nums = []
    labels = []
    mi_scores = []

    if full_exeriment:
        fname = f'{experiments_dir}/generated_grasps_experiment{experiment}/Full Experiment {experiment} log {prefix}.txt'
        
    else:
        fname = f'{experiments_dir}/generated_grasps_experiment{experiment}/Experiment {experiment} log {prefix}.txt'

        
    fp = open(fname)
    all_data_txt = fp.read().split('\n')
    fp.close()
    

    for line in all_data_txt:
        
        if len(line) < 4:
            continue
        
        if full_exeriment:
            cat, idx, env_num, label, mi_score = line.split('=')
        else:
            cat, idx, env_num, label = line.split('=')
        cats.append(cat)
        indcs.append(int(idx))
        env_nums.append(int(env_num))
        labels.append(str2array(label, dtype=int))
        if full_exeriment:
            mi_scores.append(str2array(mi_score[:-1], dtype=float))
        else:
            mi_scores.append(np.zeros_like(labels[-1]))
                
    return cats, indcs, env_nums, labels, mi_scores




def construct_df(experiment, sampler, prefix, experiments_dir, full_exeriment=True):
    
    prefix_sampler = f'{sampler}_N_N_N'

    # load isaac data
    cats, indcs, env_nums, labels, mi_scores = load_isaac_results(experiment, prefix=prefix, full_exeriment=full_exeriment)
    _, _, _, sampler_labels, sampler_mi_scores = load_isaac_results(experiment, prefix=prefix_sampler, full_exeriment=full_exeriment)

    # load npz data and sort
    init_scores = []
    final_scores = []
    exec_times = []
    init_robot_scores = []
    final_robot_scores = []

    original_scores = [] 
    sample_times = []

    for i in range(len(cats)):
        data = np.load(f'{experiments_dir}/generated_grasps_experiment{experiment}/{cats[i]}{indcs[i]:003}_{sampler}.npz')
        init_score = data[f'{prefix}_init_scores'][env_nums[i]]
        sorted_indcs = np.argsort(init_score)
        labels[i] = labels[i][sorted_indcs]
        mi_scores[i] = mi_scores[i][sorted_indcs]
        init_scores.append(init_score[sorted_indcs])
        # print(i,data[f'{prefix}_time'][env_nums[i]])
        exec_times.append(data[f'{prefix}_time'][env_nums[i]])
        final_scores.append( data[f'{prefix}_scores'][env_nums[i]][sorted_indcs])
        init_robot_scores.append( data[f'{prefix}_init_robot_scores'][env_nums[i]][sorted_indcs] )
        final_robot_scores.append( data[f'{prefix}_robot_scores'][env_nums[i]][sorted_indcs] )
        sample_times.append(data[f'{prefix_sampler}_time'][env_nums[i]])
        original_scores.append( data[f'{prefix_sampler}_original_scores'][env_nums[i]][sorted_indcs])
        sampler_labels[i] = sampler_labels[i][sorted_indcs]
        sampler_mi_scores[i] = sampler_mi_scores[i][sorted_indcs]

    # create big dataframe to work with ease
    num_grasps = len(labels[0])
    num_envs = len(np.unique(env_nums))
    
    if sampler == 'gpd':
        sample_times = np.repeat(sample_times, num_grasps)
    elif sampler == 'graspnet':
        sample_times = np.array(sample_times).flatten()
    
    df_data = {
        'cat': np.repeat(cats, num_grasps),
        'idx': np.repeat(indcs, num_grasps),
        # 'labels': np.repeat(env_nums, env_nums),
        'env_num': np.repeat(env_nums, num_grasps),
        'labels': np.concatenate(labels),
        'mi_score': np.concatenate(mi_scores),
        'init_scores': np.concatenate(init_scores),
        'final_scores': np.concatenate(final_scores),
        'init_robot_scores': np.concatenate(init_robot_scores),
        'final_robot_scores': np.concatenate(final_robot_scores),
        'exec_times': np.repeat(exec_times, num_grasps),
        'original_scores': np.concatenate(original_scores),
        'sample_times': sample_times,
        'sampler_labels': np.concatenate(sampler_labels),
        'sampler_mi_scores': np.concatenate(sampler_mi_scores)
    }
    
    print(df_data['exec_times'].shape, df_data['sample_times'].shape)
    
    df = pd.DataFrame(df_data)

    return df, num_grasps, num_envs

In [162]:
#cats, indcs, env_nums, labels, mi_scores = load_isaac_results(28, prefix="gpd_N_N_N", full_exeriment=True)

In [163]:
experiments_dir='../experiments'
sampler='graspnet'
prefix = f"{sampler}_SE_GraspFlow_Theta"
experiment=28

df, num_grasps, num_envs = construct_df(experiment, sampler, prefix, experiments_dir)

(420,) (420,)


In [165]:
df.head()

,cat,idx,env_num,labels,mi_score,init_scores,final_scores,init_robot_scores,final_robot_scores,exec_times,original_scores,sample_times,sampler_labels,sampler_mi_scores
0,bottle,3,0,4,0.014518,0.000002,0.079198,0.999904,1.0,4.965913,0.026835,0.574927,2,0.015925
1,bottle,3,0,3,0.088556,0.000025,0.000028,1.000000,1.0,4.965913,0.000172,0.574927,3,0.087002
2,bottle,3,0,1,0.044350,0.000473,0.831182,1.000000,1.0,4.965913,0.338960,0.574927,4,0.041982
3,bottle,3,0,1,0.065774,0.001095,0.983109,1.000000,1.0,4.965913,0.097442,0.574927,1,0.069279
4,bottle,3,0,4,0.097056,0.002145,0.904337,1.000000,1.0,4.965913,0.019830,0.574927,4,0.097982


In [164]:
print(f'Initial classifier score: {df.init_scores.mean()}')
print(f'After refinement score: {df.final_scores.mean()}')
print(f'Initial Robot Classifier score: {df.init_robot_scores.mean()}')
print(f'Final Robot Classifier score: {df.final_robot_scores.mean()}')
print(f'Average refinement time: {df.exec_times.mean()} ({df.exec_times.std()})')

num_unique_cats = len(df.cat.unique())
init_success_rate = df.loc[df.sampler_labels == 1, 'sampler_labels'].sum()#/(num_grasps*num_envs*num_unique_cats)
print(f'Init Success Rate: {init_success_rate}')
final_success_rate = df.loc[df.labels == 1, 'labels'].sum()#/(num_grasps*num_envs*num_unique_cats)
print(f'Final Success Rate: {final_success_rate}')

Initial classifier score: 0.17810329530789115
After refinement score: 0.6632512327994846
Initial Robot Classifier score: 0.9761695951858063
Final Robot Classifier score: 0.9952258785565697
Average refinement time: 4.57529480116708 (0.4885579125919171)
Init Success Rate: 104
Final Success Rate: 241


In [101]:
# experiment=9
# experiments_dir='../experiments'
# sampler='gpd'
# prefix = f"{sampler}_S_GraspFlow_Euler"

# df, num_grasps, num_envs = construct_df(experiment, sampler, prefix, experiments_dir)

In [ ]:
# df.head()

In [ ]:
# print(f'Initial classifier score: {df.init_scores.mean()}')
# print(f'After refinement score: {df.final_scores.mean()}')
# print(f'Initial Robot Classifier score: {df.init_robot_scores.mean()}')
# print(f'Final Robot Classifier score: {df.final_robot_scores.mean()}')
# print(f'Average refinement time: {df.exec_times.mean()} ({df.exec_times.std()})')

# num_unique_cats = len(df.cat.unique())
# final_success_rate = df.loc[df.labels == 1, 'labels'].sum()/(num_grasps*num_envs*num_unique_cats)
# print(f'Final Success Rate: {final_success_rate}')

In [ ]:
for experiment in range(9,18):
    print(f'-------------------------EXPERIMENT {experiment}---------------------------')
    experiments_dir='../experiments'
    sampler='gpd'
    prefix = f"{sampler}_S_GraspFlow_Euler"

    df, num_grasps, num_envs = construct_df(experiment, sampler, prefix, experiments_dir)

    print(f'Initial classifier score: {df.init_scores.mean()}')
    print(f'After refinement score: {df.final_scores.mean()}')
    print(f'Initial Robot Classifier score: {df.init_robot_scores.mean()}')
    print(f'Final Robot Classifier score: {df.final_robot_scores.mean()}')
    print(f'Average refinement time: {df.exec_times.mean()} ({df.exec_times.std()})')

    num_unique_cats = len(df.cat.unique())
    init_success_rate = df.loc[df.sampler_labels == 1, 'sampler_labels'].sum()#/(num_grasps*num_envs*num_unique_cats)
    print(f'Init Success Rate: {init_success_rate}')
    final_success_rate = df.loc[df.labels == 1, 'labels'].sum()#/(num_grasps*num_envs*num_unique_cats)
    print(f'Final Success Rate: {final_success_rate}')

In [ ]:
df, num_grasps, num_envs = construct_df(18, "gpd", 'gpd_S_GraspFlow_Euler', experiments_dir)
df['success'] = df.apply(lambda row: 1 if row.labels==1 else 0, axis=1)
df['init_success'] = df.apply(lambda row: 1 if row.sampler_labels==1 else 0, axis=1)

In [ ]:
print(f'Initial classifier score: {df.init_scores.mean()}')
print(f'After refinement score: {df.final_scores.mean()}')
print(f'Initial Robot Classifier score: {df.init_robot_scores.mean()}')
print(f'Final Robot Classifier score: {df.final_robot_scores.mean()}')
print(f'Average refinement time: {df.exec_times.mean()} ({df.exec_times.std()})')
print(f'Average sampling time: {df.sample_times.mean()} ({df.sample_times.std()})')

num_unique_items = df.groupby(['cat', 'idx']).count().shape[0]
init_success_rate = df.loc[df.sampler_labels == 1, 'sampler_labels'].sum()/(num_grasps*num_envs*num_unique_items)
print(f'Init Success Rate: {init_success_rate}')
final_success_rate = df.loc[df.labels == 1, 'labels'].sum()/(num_grasps*num_envs*num_unique_items)
print(f'Final Success Rate: {final_success_rate}')

In [ ]:
df_table = df[['cat', 'idx', 'init_scores', 'final_scores', 'init_success', 'success']]

In [ ]:
df_table = df_table.groupby(['cat', 'idx']).agg({'init_scores': [np.mean],
                                    'final_scores': [np.mean],
                                     'init_success': [np.sum],
                                     'success': [np.sum]}).reset_index()

In [ ]:
df_table

In [ ]:
prefixes = [
    'gpd_S_GraspFlow_Euler',
    'gpd_S_GraspFlow_Theta',
    'gpd_S_GraspFlow_SO3',
    'gpd_SE_GraspFlow_Theta',
    'gpd_S_graspnet_Euler',
    'gpd_S_metropolis_Euler'
    ]

Outline
1. Overall statistics
2. Breakdown by prefixes and objects
3. Cumulative plot
4. Breakdown of the problem

In [ ]:
full_experiment=False
all_dfs = []
for prefix in prefixes:
    df, num_grasps, num_envs = construct_df(18, "gpd", prefix, experiments_dir, full_experiment)
    df['success'] = df.apply(lambda row: 1 if row.labels==1 else 0, axis=1)
    df['init_success'] = df.apply(lambda row: 1 if row.sampler_labels==1 else 0, axis=1)
    df = df.assign(prefix=prefix)
    all_dfs.append(df)

In [ ]:
df = pd.concat(all_dfs)

#### Overall

In [ ]:
# Why GrapsFlow poor respect to graspnet method?
df_table = df[['prefix','init_scores', 'final_scores', 'success', 'init_success']]

df_table = df_table.groupby(['prefix']).agg({'init_scores': [np.mean],
                                    'final_scores': [np.mean],
                                     'init_success': [np.sum],
                                     'success': [np.sum]})

In [ ]:
df_table

In [ ]:
# Why GrapsFlow poor respect to graspnet method?
df_table = df[['prefix', 'cat', 'idx', 'init_scores', 'final_scores', 'success', 'init_success']]

df_table = df[(df.prefix == 'gpd_S_GraspFlow_Euler') | (df.prefix == 'gpd_S_graspnet_Euler')]
df_table = df_table.groupby(['prefix', 'cat', 'idx']).agg({'init_scores': [np.mean],
                                     'final_scores': [np.mean],
                                     'init_success': [np.sum],
                                     'success': [np.sum]})

In [ ]:
# Why GrapsFlow poor respect to graspnet method?
df_table = df[['prefix', 'cat', 'idx', 'init_scores', 'final_scores', 'success', 'init_success']]

df_table = df[(df.prefix == 'gpd_S_GraspFlow_Theta') | (df.prefix == 'gpd_SE_graspnet_Euler')]
df_table = df_table.groupby(['prefix', 'cat', 'idx']).agg({'init_scores': [np.mean],
                                    'final_scores': [np.mean],
                                     'init_success': [np.sum],
                                     'success': [np.sum]})

In [ ]:
df_table